In [1]:
import pandas as pd
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
import os
from datetime import datetime
import pdb;
from grocery_ml_tensorflow import GroceryML
from grocery_ml_core import GroceryMLCore
from hidden_layer_param_builder import HiddenLayerParamSetBuilder
import tensorflow as tf

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", lambda x: f"{x:.6f}")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)

print(os.getcwd())
# print("GPUs Available:", tf.config.list_physical_devices('GPU'))
#tf.debugging.set_log_device_placement(True)

# def run_all_experiments(df, model_param_sets, output_dir):
#     total = len(model_param_sets)
#     print(f"run_all_experiments() when: {datetime.now()}  output_dir: {output_dir}");
#     for index, params in enumerate(model_param_sets, 1):
#         print(f"Running Exp {index}/{total}...")
#         groceryML.run_experiment(df,  params["buildParams"], params["trainParams"], output_dir)


try:
    groceryML = GroceryML();
    groceryMLCore = GroceryMLCore();
    groceryML.build_training_df()
    if groceryML.training_df is None:
        raise();
    ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    # groceryML.training_df.to_csv(f"training_df-{ts}.csv");
except Exception as ex: 
    print(ex)
    ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    groceryML.training_df.to_csv(f"training_df-{ts}-exception.csv");


C:\Users\steve\source\repos\grocery-ml
_build_combined_df()
_build_sources()
creating target col: didBuy_target
insert_negative_samples()
compute_expected_gap_ewma()
create_item_supply_level_feat()
add_item_total_purchase_count_feat()
build_trip_interveral_feautres()
drop_rare_purchases()
validate_no_empty_columns()
self._build_combined_df() done


In [ ]:
import optuna 
def objective(trial):

    lr = trial.suggest_float("learning_rate", 0.001, 0.01)
    depth = trial.suggest_int("depth", 1, 10)
    units = trial.suggest_int("units", 5, 2048)
    embedding_dim = trial.suggest_int("embedding_dim", 300, 2048)
    epochs = trial.suggest_int("epochs", 10, 150)
    # output_activation = trial.suggest_categorical("output_activation", ["sigmoid", "linear"])
    output_activation = "sigmoid"
    metrics = ["AUC", "Precision", "Recall"]
    # metrics = ["Accuracy", "MAE", "MSE", "MAPE", "MSLE", "Precision", "Recall", "AUC",  "BinaryCrossentropy", "RootMeanSquaredError"]
    
    layers_cfg = []
    for _ in range(depth):
        layers_cfg.append({ "units": units, "activation": "relu" })

    build_params = {
        "embedding_dim": embedding_dim,
        "layers": layers_cfg,
        "output_activation": output_activation,
        "optimizer": "adam",
        "learning_rate": lr,
        "loss": "binary_crossentropy" if output_activation == "sigmoid" else "mse",
        "metrics": metrics
    }

    train_params = {
        "epochs": epochs,
        "batch_size": 32,
     
    }

    groceryML.run_experiment(
        groceryML.training_df,
        build_params,
        train_params,
        r"f:/exp/keras/optuna/after_neg_sameple_fix"
    )

    return groceryML.last_val_auc
############################################################################


sampler = optuna.samplers.TPESampler()

study_name = f"grocery_ml_tuning_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
study = optuna.create_study(
    study_name= study_name,
    sampler=sampler,
    direction="maximize",   # or minimize — see note below
    storage="sqlite:///optuna_grocery.db",
    load_if_exists=True
)


study.optimize(objective, n_trials=5000)

[I 2026-01-14 01:17:13,370] A new study created in RDB with name: grocery_ml_tuning_20260114_011712


Creating dir: f:/exp/keras/optuna/after_neg_sameple_fix\e732_l1571-1571-1571-1571-1571-
run_experiment()  exp_dir: f:/exp/keras/optuna/after_neg_sameple_fix\e732_l1571-1571-1571-1571-1571-  
run_experiment()  when: 2026-01-14 01:17:13.449558 params: {'epochs': 94, 'batch_size': 32}  
normalize_features()
train_model()
